<a href="https://colab.research.google.com/github/LarrySnyder/ASJ/blob/main/predpol/PredPol_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PredPol Simulation

This file is read-only. To work with it, you first need to **save a copy to your Google Drive:**

1. Go to the *File* menu. (The *File* menu inside the notebook, right below the filename—not the *File* menu in your browser, at the top of your screen.)
2. Choose *Save a copy in Drive*. (Log in to your Google account, if necessary.) Feel free to move it to a different folder in your Drive, if you want.
3. Colab should open up a new browser tab with your copy of the notebook. Double-click the filename at the top of the window and rename it `PredPol Simulation [your name(s)]`. 
4. Close the original read-only notebook in your browser.


---
> 👓 **Note:** This notebook is part of the *Algorithms and Social Justice* course at Lehigh University, Profs. Larry Snyder and Suzanne Edwards.
---


---
> 📚 **References:** This notebook is adapted from the PredPol simulation code by Kristian Lum and William Isaac, available at https://github.com/arun-ramamurthy/pred-pol, which implements the experiment described in their paper "[To Predict and Serve?](http://onlinelibrary.wiley.com/doi/10.1111/j.1740-9713.2016.00960.x/full)," *Significance* 13(5), 14–19, 2016.
>
>The PredPol algorithm itself is described in a research paper by Mohler, et al., "[Randomized Controlled Field Trials of Predictive Policing](https://www.jstor.org/stable/24740149)," *Journal of the American Statistical Association* 110(512), 1399–1411, 2015. See also the errata noted by Lum and Isaac [here](https://github.com/arun-ramamurthy/pred-pol/raw/master/doc/Rederivation%20of%20Mohler%20et%20al.pdf).
---



## Introduction

This notebook contains code to simulate the PredPol predictive policing algorithm and to run an experiment similar to the one described by Lum and Isaac in their paper cited above.

In a nutshell, Lum and Isaac argue that, although drug crimes are spread quite evenly across a geographical area, PredPol tends to dispatch police to a small subset of areas, and that these areas are not representative of the population as a whole:

>This suggests that while drug crimes exist
everywhere, drug arrests tend to only occur in very specific locations – the police data appear to disproportionately represent crimes committed in areas with higher populations of non-white and low-income residents.

They base their case study on Oakland, California. Because we do not know the *true* incidence of drug crimes in Oakland (or anywhere else), Lum and Isaac develop a **synthetic dataset** that approximates the incidence of illicit drug use in Oakland. (See p. 16 for more information.) This is the dataset we will use in this notebook.

Here is the synthetic dataset of drug crimes as displayed in Figure 1(b) in Lum and Isaac:

<div>
<img src="https://raw.githubusercontent.com/LarrySnyder/ASJ/main/predpol/images/LumIsaacFigure1b.png" width="600"/>
</div>

Each rectangle in the heatmap is a region of the city of Oakland (Lum and Isaac refer to these as **bins**). The PredPol algorithm uses historical data on the number of observed crimes in each bin, predicts the crime rates in each bin, and then decides which bins to dispatch police to (assuming police can be dispatched to a fixed number, say 20, of bins).

Lum and Issac simulate the PredPol algorithm using this dataset. Their Figure 2(a) displays a heatmap of the number of times the PredPol algorithm dispatched police to each bin in their simulation:

<div>
<img src="https://raw.githubusercontent.com/LarrySnyder/ASJ/main/predpol/images/LumIsaacFigure2a.png" width="600"/>
</div>

(Note the difference in the distribution of crimes compared to the distribution of police dispatches.)

In this notebook, we'll use Lum and Isaac's synthetic dataset of drug crimes in Oakland, but instead of plotting it on a map of Oakland, we'll plot it in a simple grid:

<div>
<img src="https://raw.githubusercontent.com/LarrySnyder/ASJ/main/predpol/images/sim_heatmap1.png" width="400"/>
</div>

That's because grids are way more fun, beautiful, and useful than maps. (Err...actually it's because I haven't yet learned how to do mapping in Python.)

## Preliminaries

### Importing Packages

Next we'll import the Python packages that we'll need for our analysis.

* `pandas` (pronounced like the animal) handles data
* `numpy` ("num-pie") does numerical computations
* `matplotlib` ("mat-plot-libe") and `seaborn` do graphing
* `copy` has functions for copying data structures
* `tqdm` is a progress bar

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from tqdm.notebook import tqdm

## The PredPol Algorithm

The code below implements the PredPol algorithm. You need to execute the cells, but then you can ignore them. In fact, if you want, you can "collapse" this section by clicking the little down-arrow next to the section title, to get it out of the way while you're working.

In [ ]:
# Helper functions.

def lower_tri_ij(dim):
	ii, jj = np.tril_indices(dim)
	for i, j in zip(ii, jj):
		yield i, j

def lambdafun(t, mu, theta, omega):
	days_til_last = [(t[-1] - ti).days for ti in t[:-1]]
	epart = sum([np.exp(-omega * d) for d in days_til_last])
	return mu + theta * omega * epart

def calc_tij(crime_data):
	assert all([len(n) > 0 for n in crime_data.values()])
	tij = dict()
	for n, events in crime_data.items():
		tij_len = len(events) - 1
		tij[n] = np.zeros((tij_len, tij_len))
		for i, j in lower_tri_ij(tij_len):
			td = (events[i + 1] - events[j]).days
			tij[n][i, j] = td
	return tij

def calc_pij(tij, theta, omega):
	pij = dict()
	for n in tij:
		pij[n] = np.zeros(tij[n].shape)
		for i, j in lower_tri_ij(tij[n].shape[0]):
			if tij[n][i, j] > 0:
				e_part = np.exp(-omega * tij[n][i, j])
				pij[n][i, j] = e_part * theta * omega
	return pij

In [ ]:
# Expectation step.
def estep(crime_data, mu, theta, omega, tij):
	# these asserts are fast and document the common structure
	assert crime_data.keys() == mu.keys()
	assert crime_data.keys() == tij.keys()
	pj = dict()
	pij = calc_pij(tij, theta, omega)
	for n in crime_data:
		# should possibly append a 1 to the front of this
		denom = mu[n] + pij[n].sum(axis=1)
		pj[n] = mu[n] / denom
		pij[n] = pij[n] / denom
		pj[n] = np.append(pj[n], 1)
	return pij, pj

# Maximization step.
def mstep(pij, pj, tij, crime_data, T):
    # iterating over a dict means over keys unless otherwise specified
	# these asserts are fast and document the common structure
	assert crime_data.keys() == pij.keys()
	assert crime_data.keys() == tij.keys()
	assert crime_data.keys() == pj.keys()
	total_events = sum([len(v) for v in crime_data.values()])
	# double sum: arrays over bins; np.sum whole array at once
	sum_pijs = sum([np.sum(pij[n]) for n in pij])
	denom = sum([np.sum(pij[n] * tij[n]) for n in pij])
	omega = sum_pijs / denom
	theta = sum_pijs / total_events
	mu = dict((n, sum(pj[n]) / T) for n in pj)
	# mu = np.ones(num_bins)*sum(mu)  #this is what the paper says to do but
	# this forces the "background rate" to be the same everywhere,
	return omega, theta, mu

In [ ]:
# Main PredPol function.
def runEM(crime_data, T, pred_date, k=20,
		  theta_init=1, omega_init=1, mu_init=1,
		  tol1=.00001, tol2=.00001, tol3=.0001):
	num_bins = len(crime_data)
	theta = theta_init
	mu = dict((key, mu_init) for key in crime_data)
	omega = omega_init
	omega_last = 10 + omega
	theta_last = 10 + theta
	mu_last = dict((key, mu_init + 10) for key in crime_data)
	k = min(num_bins, k)
	tij = calc_tij(crime_data)

	while(abs(omega - omega_last) > tol1 and
		  abs(theta - theta_last) > tol2 and
		  sum([abs(mu[n] - mu_last[n]) for n in mu]) > tol3):
		omega_last = omega
		theta_last = theta
		mu_last = copy.deepcopy(mu)
		pij, pj = estep(crime_data, mu, theta, omega, tij)
		omega, theta, mu = mstep(pij, pj, tij, crime_data, T)

		assert omega < T * 1000

	# get conditional intensity for selected parameters
	# need to add on latest date
	for n in crime_data:
		crime_data[n] = np.append(crime_data[n], pred_date)

	# Calculate rate for each bin.
	rates = {n: lambdafun(crime_data[n], mu[n], theta, omega) for n in crime_data}

	return rates, omega, theta

## Plotting

The next function plots heatmaps to visualize the results of the simulation. Again, execute the cell and then you can collapse this section. 

In [ ]:
def plot_heatmaps(observed_filepath, flagged_filepath, output_pdf_filepath=None):
	"""Plot heatmaps. Based on the results of a completed PredPol run."""

	# Read CSV files.
	observed_df = pd.read_csv(observed_filepath, index_col='bin')
	flagged_df = pd.read_csv(flagged_filepath, index_col='bin')

	# Get # of crimes in each bin.
	observed_crimes = observed_df.sum(axis=1)
	observed_crimes_array = np.array(list(observed_crimes))

	# Get # of times each bin is flagged.
	flags = flagged_df.sum(axis=1)
	flags_array = np.array(list(flags))

	# Get shortcut to # of columns.
	num_cols = 20

	# Add 0s so that size of arrays are multiples of num_cols.
	while len(observed_crimes_array) % num_cols != 0:
		observed_crimes_array = np.append(observed_crimes_array, [0])
	while len(flags_array) % num_cols != 0:
		flags_array = np.append(flags_array, [0])

	# Convert to 10-column numpy arrays.
	observed_crimes_array = observed_crimes_array.reshape(-1, num_cols)
	flags_array = flags_array.reshape(-1, num_cols)

	# Build heatmaps.
	fig, axes = plt.subplots(1, 2)
	fig.set_figheight(5)
	fig.set_figwidth(12)
	sns.heatmap(observed_crimes_array, cmap='flare', ax=axes[0])
	axes[0].title.set_text('Observed Crimes')
	sns.heatmap(flags_array, cmap='flare', ax=axes[1])
	axes[1].title.set_text('# Times Flagged')

	# Title.
	fig.suptitle(f"(total crimes observed: {observed_df.sum().sum()})")

	if output_pdf_filepath:
		fig.savefig(output_pdf_filepath, bbox_inches='tight')

	plt.show()

## PredPol Simulation: Helper Functions

The next few functions do some processing that is required for the simulation and the PredPol algorithm. Execute and collapse!

In [ ]:
def load_predpol_data(options):
	"""Load crime data from CSV file at URL specified by `options`."""

	# Read crime data. 
	crime_df = pd.read_csv(options["input_data_url"])
	# Specify column names.
	crime_df.columns = ['rownum', 'bin', 'date_occurred', 'LAG']
	# Drop rownum and 'LAG' columns.
	crime_df = crime_df.drop(['rownum', 'LAG'], axis=1)
	# Remove rows with null bins.
	crime_df = crime_df[pd.notnull(crime_df['bin'])]
    # Format bins as integer.
	crime_df['bin'] = crime_df['bin'].astype(int)
	# Format dates.
	crime_df['date_occurred'] = pd.to_datetime(crime_df.date_occurred, format='%m/%d/%y')

	return crime_df

In [ ]:
def prepare_data_for_predpol(crime_data, start_date, end_date):
	"""Prepare crime_data for PredPol simulation."""

	# Remove records that are not in the desired time window.
	crime_data = crime_data[(crime_data.date_occurred >= start_date) & (crime_data.date_occurred < end_date)]

	# Build output dict. (Keys are map bins, values are crime_data values.)
	pp_dict = dict((int(i), []) for i in set(crime_data.bin))
	for i in crime_data.index:
		pp_dict[int(crime_data.bin[i])].append(crime_data['date_occurred'].loc[i])

	# Drop empty rows.
	keys_to_drop = []
	pp_dict = {n: sorted(v) for n, v in pp_dict.items()}
	for n in pp_dict.keys():
		if len(pp_dict[n]) < 1:
			keys_to_drop.append(n)
	for key in keys_to_drop:
		pp_dict.pop(key, None)

	return pp_dict

In [ ]:
def get_empty_bin_dataframe(max_bin):
	"""Build an empty dataframe with index = 'bin' and indices from 1, ..., max_bin. """

	df = pd.DataFrame()
	df['bin'] = range(1, max_bin + 1)
	df = df.set_index(['bin'])

	return df

In [ ]:
def add_column_to_bin_dataframe(df, bins, values, str_date):
	"""Add a column (in place) to `df`, with header `str_date`. For each bin in `bins`, the function
	sets its value equal to the corresponding value in `values`; all other values are set to 0. 
	(`bins` and `values` must be lists of the same length.)
	This function takes care of a few annoying aspects of adding the column, e.g., 
	handling first column vs. subsequent ones."""
	temp_series = pd.DataFrame(0, index=df.index, columns=[str_date])
	temp_series.loc[bins, str_date] = values
	if df.empty:
		df[str_date] = list(temp_series[str_date])
	else:
		df = pd.concat([df, temp_series], axis=1)
	return df

## PredPol Simulation: Main Logic

This section contains the main code for the PredPol simulation. The first function processes the data to feed into the PredPol algorithm and then calls the algorithm itself. It returns a variable called `rates`, which contains the crime rate predicted by PredPol for each bin.

In [ ]:
def do_predpol_calculations(crime_data_df, flagged_df, window_start, window_end, options):
	"""Run PredPol algorithm. Return rates, as well as crime_data_df in case it
	changed within this function."""

	# Prepare data for PredPol calculations.
	pp_dict = prepare_data_for_predpol(crime_data_df, window_start, window_end)

	# Run PredPol calculations.
	rates, _, _ = runEM(pp_dict, options["predpol_window"], window_end)

	return rates, crime_data_df

The next function takes as input the `rates` predicted by the PredPol algorithm and returns as output the bins that are flagged for increased police presence today. To decide which bins to flag, the function simply chooses the bins with the largest predicted rates, but you can change this code later, if you want, to try different methods.

In [ ]:
def choose_flagged_bins(rates, observed_crimes_df, num_flags=20):
	"""Choose which bins to flag, given the bins and their rates."""

	# Sort rates.
	sorted_bins = sorted(rates, key=lambda x: rates[x], reverse=True)
	sorted_rates = [rates[b] for b in sorted_bins]

	# Choose the num_flags bins with the largest rates.
	flagged_bins = sorted_bins[0:num_flags]

	return flagged_bins

Next, we have the main function for the simulation. This function does some preliminary processing, then iterates through the days of the simulation; on each day, it runs the PredPol algorithm, chooses which bins to send additional police to, adds 20% more crimes to those bins, and updates the data structures.

In [ ]:
def run_predpol_simulation(crime_data_df, options):
	"""Run the PredPol simulation."""

	# Determine number of bins needed for dataframe.
	max_bin = int(max(crime_data_df.bin))

	# Remove data that are not in the desired time window.
	crime_data_df = crime_data_df[(crime_data_df.date_occurred >= options["simulation_start"]) & (crime_data_df.date_occurred < options["simulation_end"])]

	# Make a copy to serve as basline (just for record-keeping).
	baseline_crimes_df = crime_data_df.copy(deep=True) 

	# Determine number of days to simulate.
	num_days = (options["simulation_end"] - options["simulation_start"]).days - options["predpol_window"]
	
	# Initialize dataframe for PredPol rates, observed crimes, and whether a bin was flagged.
	predpol_rates_df = get_empty_bin_dataframe(max_bin)
	observed_crimes_df = get_empty_bin_dataframe(max_bin)
	flagged_df = get_empty_bin_dataframe(max_bin)

	# Initialize heatmap figure.
	fig = None

	# Initialize progress bar.
	pbar = tqdm(total=num_days)

	# Loop through days in simulation.
	for i in range(num_days):

		# Update progress bar.
		pbar.update()

		# Determine dates of PredPol window.
		window_start = options["simulation_start"] + pd.DateOffset(i)
		window_end = options["simulation_start"] + pd.DateOffset(i + options["predpol_window"])

		# Run PredPol.
		rates, crime_data_df = do_predpol_calculations(crime_data_df, flagged_df, window_start, window_end, options)

		# Choose which bins to send police to.
		flagged_bins = choose_flagged_bins(rates, observed_crimes_df, options["num_flags"])

		# Get today's baseline crimes.
		crime_today = pd.value_counts(crime_data_df[crime_data_df.date_occurred==window_end].bin)

		# Save today's data.
		str_date = str(window_end).split(' ')[0]
		predpol_rates_df = add_column_to_bin_dataframe(predpol_rates_df, bins=list(rates.keys()), values=list(rates.values()), str_date=str_date)
		flagged_df = add_column_to_bin_dataframe(flagged_df, bins=flagged_bins, values=[1] * len(flagged_bins), str_date=str_date)
		observed_crimes_df = add_column_to_bin_dataframe(observed_crimes_df, bins=crime_today.index, values=list(crime_today), str_date=str_date)

		# Remove old data from crime_data_df to speed things up.
		crime_data_df = crime_data_df[(crime_data_df.date_occurred >= window_start)]

	# Close progress bar.
	pbar.close()

	# Write results to CSV.
	predpol_rates_df.to_csv(options["rates_filename"])
	flagged_df.to_csv(options["flagged_filename"])
	baseline_crimes_df.to_csv(options["baseline_crime_filename"])
	observed_crimes_df.to_csv(options["observed_crime_filename"])

## Running the PredPol Simulation

Finally, we can run the PredPol simulation. 

### Options

First we'll specify various options for running the simulation.

In [ ]:
# Initialize the `options` dictionary.
options = {}

In [ ]:
# The URL of the comma-separated value (CSV) file containing the synthetic crime dataset.
options["input_data_url"] = "https://raw.githubusercontent.com/LarrySnyder/ASJ/main/predpol/input/drug_crimes_with_bins.csv"

In [ ]:
# Start and end dates for the simulation.
# (Tip: Leave the start date set to 2010/07/01. For the full simulation, set
# the end date to 2011/12/31. For a faster simulation (e.g., for debugging),
# choose an earlier end date.)
options["simulation_start"] = pd.to_datetime("2010/07/01")
options["simulation_end"] = pd.to_datetime("2011/12/31")	

In [ ]:
# Paths where we want to store the output files.
options["baseline_crime_filename"] = "predpol_drug_baseline.csv"
options["observed_crime_filename"] = "predpol_drug_observed.csv"
options["rates_filename"] = "predpol_drug_rates.csv"
options["flagged_filename"] = "predpol_flagged.csv"
options["heatmap_filename"] = "heatmap.pdf"

In [ ]:
# Prediction window for PredPol. This must be larger than the number of days
# between the start and end times of the simulation.
# (Lum and Isaac use 180, but the simulation is faster (though less accurate) 
# when we use a smaller window.)
options["predpol_window"] = 120

In [ ]:
# Number of bins to flag each day.
options["num_flags"] = 20

### Loading the Data

Next, we'll load the data into a `pandas` dataframe (a structure for storing large amounts of data—like a spreadsheet or database).

In [ ]:
# Load the data.
crime_df = load_predpol_data(options)

In [ ]:
# Print the head, i.e., the first batch of rows.
crime_df.head()

The dataframe contains 2 columns:

* `bin` contains the index of the bin where the crime was observed
* `date_occurred` contains the date the crime was observed

So, the 5 records displayed in the head indicate that 3 crimes were observed in bin 67 on January 8, 2012; and one crime each was observed in bins 219 and 424 on December 31, 2011.

Let's interview the data a bit more:

In [ ]:
# How many total crimes are contained in the dataset?
print(f"Number of crimes contained in dataset = {len(crime_df)}")

In [ ]:
# How many bins are there? (The bins are indexed consecutively, so this
# is the same as asking what is the largest bin index.)
print(f"Number of bins = {max(crime_df['bin'])}")

In [ ]:
# Count the number of crimes (across all days) in each bin.
crimes_per_bin = crime_df['bin'].value_counts()

In [ ]:
# How many crimes were reported in bin 140 (across all days)?
print(f"Number of crimes in bin 140 = {crimes_per_bin[140]}")

In [ ]:
# What's the largest number of crimes (across all days) in a single bin? 
print(f"Largest number of crimes per bin = {max(crimes_per_bin)}")

In [ ]:
# Plot a histogram of the number of crimes per bin.
hist = crimes_per_bin.hist(bins=20)
plt.xlabel('Number of crimes per bin')
plt.ylabel('Number of bins');

From the histogram, we can see that most bins (>250 of them) have fewer than 50 or so crimes, while a few have many more; for example, one bin has roughly 600 crimes.

### Questions

(Feel free to add code blocks as needed to answer these questions.)

**Question 1a**

How many crimes were reported in bin 42 (across all days)?

**Answer:** *YOUR ANSWER HERE*

**Question 1b**

What is the smallest number of crimes (across all days) in a single bin?

**Answer:** *YOUR ANSWER HERE*

**Question 1c (optional, bonus*)**

(*No actual extra credit. But we'll be super proud of you.)

Plot the total number of crimes on each day (across all bins). That is, plot a graph in which the x-axis indicates the days and the y-axis indicates the number of crimes on each day.

*Hints*:

* In the code above, we built an object (`crimes_per_bin`) that gives the total number of crimes in each bin (across all days). Could you instead build an object (maybe called `crimes_per_day`) that gives the total number of crimes on each day (across all bins)?
* We built a histogram of the crimes per bin. A histogram tells you the total number of objects in each category (e.g., total number of bins with each crime count). Here, we don't want a histogram, we want a more straightforward plot. The `plot()` function in `pandas` can do this for you.

## Running the Simulation

Next, we run the simulation. Depending on the `options` you have set, this might take a minute or two.

*Hint*: If you want to run a faster simulation, try choosing an earlier date for `options["simulation_end"]`. You won't be running the simulation for the whole dataset, but sometimes we need to run a "toy" version of the data while we're experimenting or testing, and before we run the full dataset.

In [ ]:
# Run the simulation.
run_predpol_simulation(crime_df, options)

## Visualizing the Results

In [ ]:
# Plot heatmaps to visualize the results.
plot_heatmaps(
    observed_filepath=options["observed_crime_filename"],
    flagged_filepath=options["flagged_filename"],
    output_pdf_filepath=options["heatmap_filename"]
)

The first heatmap plots the number of observed crimes in each bin, which includes both the "baseline" crimes (those that would be observed regardless of where we send police) as well as the extra crimes that the police observe in the bins they are dispatched to.

The second heatmap plots the number of times each bin is flagged, i.e., the number of times police are dispatched to each bin. 

(These heatmaps are analogous to Figures 1(b) and 2(a), respectively, in Lum and Isaac.)

### Questions

**Question 2**

What conclusions do you draw from these two heatmaps? Does this policing strategy seem fair? Why or why not?

**Answer:** *YOUR ANSWER HERE*

**Question 3**

Propose a different way to assign police to bins that you feel is more fair than the current strategy. Write a paragraph that discusses:

* How you are defining "fairness" (perhaps in terms of the heatmaps, or other outputs of the model—your definition can be qualitative rather than quantitative if you wish)
* What your strategy is
* Why you feel your strategy will result in a more fair assignment of police to bins, under your definition of fairness

*Important*: There is more than one way to answer this question! We're asking you to think critically and creatively here, not to lead you to some "correct" answer.

**Answer:** *YOUR ANSWER HERE*

**Question 4 (optional, mega bonus*)** 

(*See above.)

Modify the simulation code above to implement your proposed dispatch strategy. (*Hint*: Your implementation should probably happen in the `choose_flagged_bins()` function.)

We're here to help, if you need us!

**Answer:** *YOUR ANSWER HERE*

---

## How to Submit this Notebook

* Please click the "Share" button at the top of the page. In the Share options:

    * Under "General Access", change to "Restricted" (instead of "Anyone with the link").
    * At the top, share it with Oumaima (ous219@lehigh.edu), Larry (lvs2@lehigh.edu), and Suzanne (sme6@lehigh.edu).
    * Click "Copy Link".
    * Click "Done".

* Send a Slack DM to Oumaima, Larry, Suzanne, and your partner with the link you copied.
